## PSO compression search on LeNet-5 

In [1]:
PATH_PREFIX = '../../'
import sys
sys.path.append(PATH_PREFIX)

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import os
import math

from data.mnist import MnistDataset
from data.utils.mnist_utils import *
from models.lenet.lenet import LeNet5
from utils.weight_sharing import *
from utils.pso import PSOController, Particle
from utils.plot import *
from utils.fitness_controller import FitnessController

Setting parameters

In [3]:
# net train settings
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
N_CLASSES = 10
DEVICE = 'cpu'
EPOCHS = 100

# net save settings
NET_TYPE = 'relu'
NET_PATH = os.path.join(PATH_PREFIX, f'models/lenet/saves/lenet_{NET_TYPE}.save')

# dataset settings
DATA_PATH = os.path.join(PATH_PREFIX, 'data')

# pso iter count
NUM_ITERATIONS = 20
NUM_PARTICLES = 20

# pso search settings
PARTICLE_REPR_RANGES = [range(1, 51) for _ in range(5)]
PARTICLE_MAX_VELOCITY = [4 for _ in range(5)]
INERTIA = 0.8

# pso save settings
SAVE_PSO_FILE = os.path.join(PATH_PREFIX, 'results/test_PSO_save.csv')
SAVE_EVERY = 1

# bh settings
BH_RADUIUS = None #2
BH_REPR_RAD = False
BH_VEL_TRESHOLD = None #2

# target position
TARGET = [1.0, 12.0]
TARGET_LOW_LIMIT = [0.95, 1.0]
LOCK_TARGET = False
TARGET_UPDATE_OFFSET = [0.001, 0.1]
TOP_REPR_SET_INDIV = True

# WS settings
SHARE_ORDER = [0, 1, 2, 3, 4]
RETRAIN_AMOUNT = None #[0, 0, 0, 0, 0]
PREC_REDUCT = ['f4', 'f4', 'f4', 'f4', 'f4']
CLUST_MOD_FOCUS = None #[0, 0, 0, 0, 0]
CLUST_MOD_SPREAD = None #[0, 0, 0, 0, 0]
MINI_BATCH_KMEANS = True

# range optimization settings
RANGE_OPTIMIZATION = True
RANGE_OPTIMIZATION_TRESHOLD = 0.97
RANGE_OPTIMIZATION_FILE = os.path.join(PATH_PREFIX, f'models/lenet/saves/lenet_{NET_TYPE}_layer_perf_{PREC_REDUCT[0]}.csv')

Geting somewhat trained LeNet-5

In [4]:
dataset = MnistDataset(BATCH_SIZE, DATA_PATH, val_split=0.5)
model = LeNet5(N_CLASSES, NET_TYPE)
model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
train_settings = [criterion, optimizer, dataset, EPOCHS, DEVICE, 1, True]

get_trained(model, NET_PATH, train_settings, DEVICE)

before_loss = get_accuracy(model, dataset.test_dl, DEVICE)

In [5]:
lam_opt = lambda mod : torch.optim.Adam(mod.parameters(), lr=LEARNING_RATE)
lam_train = lambda opt, epochs : train_net(model, criterion, opt, dataset, epochs, device=DEVICE)
lam_test = lambda : get_accuracy(model, dataset.test_dl, DEVICE)

ws_controller = WeightShare(model, lam_test, lam_opt, lam_train)
ws_controller.set_reset()

ws_controller.print_layers_info()

layer_name #weights #bias w_locked CR
feature_extractor.0 150 6 False 1.00
feature_extractor.3 2400 16 False 1.00
feature_extractor.6 48000 120 False 1.00
classifier.0 10080 84 False 1.00
classifier.2 840 10 False 1.00
Sum num weights, bias:  61470 236
Compression rate 1.00


Defining fitness function

In [6]:
def fitness_vals_fc(individual:Particle):
    # reset the net
    ws_controller.reset()
    
    # share weigts by particle
    if individual.data is None:
        individual.data = ws_controller.share(individual.representation, SHARE_ORDER, RETRAIN_AMOUNT, PREC_REDUCT, CLUST_MOD_FOCUS, CLUST_MOD_SPREAD, minibatch_kmeans=MINI_BATCH_KMEANS)
    
    return [individual.data['accuracy'], individual.data['compression']]

def fit_from_vals(data, targ_vals):

    return 1 / math.sqrt(pow(1 - (data['accuracy']/targ_vals[0]), 2) + pow(1 - (data['compression']/targ_vals[1]), 2))

Define logging function

In [7]:
data = {
    'time': [],
    'position': [],
    'representation': [],
    'velocity': [],
    'accuracy': [],
    'accuracy_loss': [],
    'compression': [],
    'share_t': [],
    'train_t': [],
    'acc_t': []
}

data_types = {
    'time' : 'uint8',
    'accuracy': 'float32',
    'accuracy_loss': 'float32',
    'compression': 'float32',
    'share_t': 'float32',
    'train_t': 'float32',
    'acc_t': 'float32'
}

pso_data = pd.read_csv(SAVE_PSO_FILE).astype(data_types) if os.path.exists(SAVE_PSO_FILE) else pd.DataFrame(data).astype(data_types)

def logger_fc(pso_cont:PSOController):
    global pso_data

    new_data = copy.deepcopy(data)

    for particle in pso_cont.swarm:

        new_data['time'].append(pso_cont.time)
        new_data['position'].append(particle.position)
        new_data['representation'].append(particle.representation)
        new_data['velocity'].append(particle.velocity)
        new_data['accuracy'].append(particle.data['accuracy'])
        new_data['accuracy_loss'].append(before_loss - particle.data['accuracy'])
        new_data['compression'].append(particle.data['compression'])
        new_data['share_t'].append(particle.data['times']['share'])
        new_data['train_t'].append(particle.data['times']['train'])
        new_data['acc_t'].append(particle.data['times']['test'])

    # saving progress
    pso_data = pso_data.append(pd.DataFrame(new_data).astype(data_types))
    if pso_cont.time % SAVE_EVERY == SAVE_EVERY - 1:
        pso_data.reset_index(drop=True, inplace=True)
        os.makedirs(os.path.dirname(SAVE_PSO_FILE), exist_ok=True)
        pso_data.to_csv(SAVE_PSO_FILE, index=False)

Optimizing ranges

In [8]:
lam_test_inp = lambda _ : get_accuracy(model, dataset.test_dl, DEVICE)

if RANGE_OPTIMIZATION:
    PARTICLE_REPR_RANGES = ws_controller.get_optimized_layer_ranges(PARTICLE_REPR_RANGES, lam_test_inp, RANGE_OPTIMIZATION_TRESHOLD, 
        savefile=RANGE_OPTIMIZATION_FILE)

for repr_range in PARTICLE_REPR_RANGES:
    print(len(repr_range))

49
47
47
48
47


Running the search

In [9]:
fit_controll = FitnessController(TARGET, fitness_vals_fc, fit_from_vals, target_update_offset=TARGET_UPDATE_OFFSET, 
    lock=LOCK_TARGET, target_limit=TARGET_LOW_LIMIT)
pso = PSOController(NUM_PARTICLES, PARTICLE_REPR_RANGES, PARTICLE_MAX_VELOCITY, INERTIA, fit_controll, 
    BH_radius=BH_RADUIUS, BH_vel_tresh=BH_VEL_TRESHOLD, BH_repr_rad=BH_REPR_RAD)

if pso_data.size != 0:
    pso.load_from_pd(pso_data, verbose=True)
elif TOP_REPR_SET_INDIV:
    pso.swarm[0].set_pos([float(len(rng)) for rng in PARTICLE_REPR_RANGES])

pso.run(NUM_ITERATIONS, logger_fc, verbose=True)

Time 1/20 (0) best fitness 3.331262482753833


KeyboardInterrupt: 

In [ ]:
pso_data.tail()

In [ ]:
plot_alcr(pso_data)
plt.title('PSO algorithm on LeNet-5')